In [526]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import font_manager, rc

In [527]:
#경쟁업종 리스트 제작
competeStore = ["닭/오리요리", "한식", "일식/수산물", "양식", "분식", "패스트푸드", "부페", "기타음식업", "제과제빵떡케익", "별식/퓨전요리", "음식배달서비스", "유흥주점", "커피점/카페", "중식", "광고/인쇄", "개인/가정용품수리", "자동차/이륜차", "주유소/충전소", "주택수리", "운송/배달/택배", "예식/의례/관혼상제", "기타서비스업", "대중목욕탕/휴게", "세탁/가사서비스", "물품기기대여", "법무세무회계", "개인서비스", "인력/고용/용역알선", "대행업", "사진", "행사/이벤트", "이/미용/건강", "장례/묘지", "종합소매점", "중고품소매/교환", "의복의류", "기타판매업", "음/식료품소매", "가정/주방/인테리어", "선물/팬시/기념품", "사무/문구/컴퓨터", "가방/신발/액세서리", "애견/애완/동물", "유아용품", "자동차/자동차용품", "의약/의료품소매", "화장품소매", "사진/광학/정밀기기소매", "가전제품소매", "시계/귀금속소매", "종교용품판매", "운동/경기용품소매", "취미/오락관련소매", "예술품/골동품/수석/분재", "가구소매", "철물/난방/건설자재소매", "페인트/유리제품소매", "책/서적/도서", "건강/미용식품", "전시/관람", "놀이/여가/취미", "요가/단전/마사지", "PC/오락/당구/볼링등", "스포츠/운동", "경마/경륜/성인오락", "무도/유흥/가무", "연극/영화/극장", "실내운동시설", "실외운동시설", "운영관리시설", "학원-자격/국가고시", "유아교육", "학교", "특수교육기관", "기타교육기관", "도서관/독서실", "연구소", "학원기타", "학원-예능취미체육", "학원-창업취업취미", "학원-보습교습입시", "학원-음악미술무용", "학원-어학", "학문교육기타", "학원-컴퓨터", "모텔/여관/여인숙", "캠프/별장/펜션", "유스호스텔", "호텔/콘도", "민박/하숙", "부동산임대", "분양", "부동산중개", "평가/개발/관리", "부동산관련서비스"]
numa = 1
resultData = pd.DataFrame(columns=range(2))

In [528]:
#CSV 파일 읽어와서 데이터프레임 만들기
compete = pd.read_csv("경쟁업종.csv")
landPrice = pd.read_csv("공시지가.csv")
population = pd.read_csv("유동인구.csv")

In [529]:
#데이터프레임 안에서 최대값과 구 구해서 데이타프로그램으로 저장하기
def getMaxPriceElement(data):
    maxValue = max(data.iloc[:,1])
    da = data.loc[(data.iloc[:,1]== maxValue), :]
    return da['구'].astype(str)

In [530]:
#그래프 제작해서 저장하기
def WriteGraph(dataFrame, store) :
    rc('font', family='HCR Dotum')
    x = range(25)
    plt.xlabel("자치구")
    plt.ylabel(store)
    plt.bar(x,dataFrame['수'])
    plt.xticks(x,dataFrame['구'],rotation=90)
    plt.savefig('my_plot'+str(numa)+'.png',bbox_inches='tight', pad_inches=0.5)
    plt.cla()

In [531]:
# 데이터프레임 안에서 최대값 구하기
def SearchMax(dataFrame) :
    Max=0
    for data in dataFrame['수']:
        if data >= Max :
            Max=data
    
    return Max

In [532]:
# 데이터프레임 안에서 최솟값 구하기
def SearchMin(dataFrame) :
    Min=dataFrame['수'][0]
    for data in dataFrame['수']:
        if data <= Min :
            Min=data
    return Min

In [533]:
#지표(0~10)계산
def IndicationCal(dataFrame) :
    mindata = SearchMin(dataFrame)
    maxdata = SearchMax(dataFrame)
    number = 0
    dataFrame = dataFrame.astype({'수':float})
    for data in dataFrame['수']:
        dataFrame['수'][number] = round((data-mindata)/(maxdata-mindata)*10,3)
        number += 1
    return dataFrame

In [534]:
#지표(0~10)계산 역방향
def IndicationCalReverse(dataFrame) :
    mindata = SearchMin(dataFrame)
    maxdata = SearchMax(dataFrame)
    number = 0
    dataFrame = dataFrame.astype({'수':float})
    for data in dataFrame['수']:
        dataFrame['수'][number] = round(10-((data-mindata)/(maxdata-mindata)*10),3)
        number += 1
    return dataFrame

In [535]:
#업종에 따른 모든 구의 지표값*가중치 값 
def WeightSet(floatPopulation, competeSector, officialPrice) :
    f_p = IndicationCal(floatPopulation)
    c_s = IndicationCalReverse(competeSector)
    o_p = IndicationCalReverse(officialPrice)
    number_f = 0
    recoList = pd.DataFrame(columns = ['구' , '수'])
    for state_f in f_p['구']:
        number_c = 0
        number_o = 0
        sumdata = 0.0
        sumdata += (f_p['수'][number_f]*0.3)
        for state_c in c_s['구']:
            if state_f == state_c :
                sumdata += (c_s['수'][number_c]*0.4)
            number_c += 1
        for state_o in o_p['구']:
            if state_f == state_o :
                sumdata += (o_p['수'][number_o]*0.3)
            number_o += 1
        recoList.loc[number_f]=[state_f, sumdata]
        number_f += 1
    return recoList

In [537]:
#업종별 데이터프레임 만들기
def SectorData(dataFrame) :
    setStore = "닭/오리요리"
    stateT = []
    dataT = []
    global store1
    for state, store, data in zip(dataFrame["구"], dataFrame["업종"], dataFrame["수"]) :
        if setStore != store :
            df = pd.DataFrame(columns=['구', '수'])
            for st, nu in zip(stateT, dataT) :
                df.loc[len(df)] = [st, nu]
            #WriteGraph(df, setStore)
            k = WeightSet(population, df, landPrice)
            global numa
            numa += 1
            price = [getMaxPriceElement(k), setStore]
            resultData.loc[len(resultData)] = price
            setStore = store
            stateT = []
            dataT = []
        stateT.append(state), dataT.append(data)
    return resultData     
dd = SectorData(compete)
dd 

<ipython-input-533-7ef99ed761ff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame['수'][number] = round((data-mindata)/(maxdata-mindata)*10,3)
<ipython-input-533-7ef99ed761ff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame['수'][number] = round((data-mindata)/(maxdata-mindata)*10,3)
<ipython-input-533-7ef99ed761ff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame['수'][number] = round((data-mindata)/(maxdat

<ipython-input-533-7ef99ed761ff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame['수'][number] = round((data-mindata)/(maxdata-mindata)*10,3)
<ipython-input-533-7ef99ed761ff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame['수'][number] = round((data-mindata)/(maxdata-mindata)*10,3)
<ipython-input-533-7ef99ed761ff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame['수'][number] = round((data-mindata)/(maxdat

<ipython-input-533-7ef99ed761ff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame['수'][number] = round((data-mindata)/(maxdata-mindata)*10,3)
<ipython-input-533-7ef99ed761ff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame['수'][number] = round((data-mindata)/(maxdata-mindata)*10,3)
<ipython-input-533-7ef99ed761ff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame['수'][number] = round((data-mindata)/(maxdat

,0,1
0,"15 성동구 Name: 구, dtype: object",닭/오리요리
1,"6 구로구 Name: 구, dtype: object",한식
2,"6 구로구 Name: 구, dtype: object",일식/수산물
3,"6 구로구 Name: 구, dtype: object",양식
4,"2 강북구 Name: 구, dtype: object",분식
...,...,...
145,"6 구로구 Name: 구, dtype: object",학원-자격/국가고시
146,"2 강북구 Name: 구, dtype: object",유아교육
147,"6 구로구 Name: 구, dtype: object",도서관/독서실
148,"6 구로구 Name: 구, dtype: object",학원기타


In [540]:
dd.to_csv('총데이터.txt')